# Market Approach 측정

In [1]:
# 의사록 읽기
import pandas as pd
minutes_ls = pd.read_csv("./data/minutes/minutes_preprocessed.csv").fillna('')

df = pd.read_csv('./data/model/bok_dictionary.csv', index_col='Unnamed: 0')
dic = df.to_dict('index')

In [2]:
index = 1
minutes_ls['Foreign Currency'][index]+'@@@'+minutes_ls['Financial Markets'][index]

'일부 위원은 6월 들어 국제유가가 크게   오르고 원화와 엔화간 동조화 현상이 뚜렷해지면서 고유가에 상대적으로 취약한   일본과 우리나라의 화폐가치가 약세를 시현하고 있어 당초 예상보다 장기간 환율  상승이 이어질 가능성은 없는지 물었으며, 이에 대해 관련부서에서는 고유가는 현  재 엔화와 원화 가치의 절하압력으로 작용하고 있으나 환율에 영향을 미치는 주  요 요인들의 비중이 시간이 지남에 따라 달라지고 있음을 감안할 때 고유가가 더   이상 원화 약세요인으로 작용하지 못할 가능성도 배제할 수 없다고 설명하였음.@@@다른 일부 위원은 최근 원/달러 환율의 급등 요인으로서 비거주자의 차액결  제선물환(NDF) 매입이 단기간에 크게 늘고 국내은행의 외환매입세가 가세하게 된   이유를 물었으며, 이에 대해 관련부서에서는 비거주자의 NDF 매입 급증은 일정   범위를 정해놓고 이를 넘어설 경우 자동적으로 매매시스템을 가동시키는 모델펀  드에 의해 촉발되었으며 글로벌 헤지펀드들이 거래에서 이익을 얻을 수 있다는   판단 하에 매입에 나선 데다 국내 역외펀드들의 국내 주식투자자금 헤지를 위한   수요에도 기인하며, 국내은행들의 매입세 가세는 수출대금 네고는 미루고(lag) 수  입 결제자금 환전은 앞당기는(lead) 무역업체들의 결제행태가 반영되는 한편 은행  들이 환율상승을 예상하여 외환매입초과 포지션(long position)을 유지하고 있는 데   기인한다고 설명하였음.@@@동 위원은 6월중 자본 순유출규모가 전월에 비해 다소 축소되었으나 항목  별로 볼 때 해외증권투자는 증가세를 지속하여 상반기 누계액이 전년동기 실적을     - 5 -  크게 상회하고 있다면서 하반기에도 해외증권투자의 증가세가 지속될 것으로 보  는지 물었으며, 이에 대해 관련부서에서는 한국은행이 외환보유액 활용 차원에서   실시하고 있는 금융기관과의 외화대출 연계 통화스왑 등에 따라 해외증권투자는   늘어날 것으로 예상된다고 답변하였음.@@@또 다른 일부 위원은 국제투자은행들이 하반기에 내외

In [3]:
import numpy as np
import pandas as pd

df = pd.DataFrame(columns=['date','tone','no_hawkish_tone_s', 'no_dovish_tone_s', 'tones', 'no_hawkish_features', 'no_dovish_features', 'hawkish_features', 'dovish_features' ])

for i, minutes in minutes_ls.iterrows() :
    hawkish_features_ls = []
    dovish_features_ls = []
    no_hawkish_features_ls = []
    no_dovish_features_ls = []
    tone_s_ls = []
    no_hawkish_tone_s = no_dovish_tone_s = tone = 0

    sentences = minutes["ngram"].split('@@@')
    
    for sentence in sentences :
        no_hawkish_features = no_dovish_features = tone_s = 0
        hawkish_features = []
        dovish_features = []
        for token in sentence.split(',') :
            if token in dic.keys() :
                if dic[token]['lable'] == 1.0 :
                    no_hawkish_features += 1
                    hawkish_features.append(token)
                if dic[token]['lable'] == -1.0 :
                    no_dovish_features += 1
                    dovish_features.append(token)
                    
        hawkish_features_ls.append(','.join(hawkish_features))
        dovish_features_ls.append(','.join(dovish_features))

        if no_hawkish_features+no_dovish_features > 0 :
            tone_s = (no_hawkish_features-no_dovish_features)/(no_hawkish_features+no_dovish_features)
    
        no_hawkish_features_ls.append(no_hawkish_features)
        no_dovish_features_ls.append(no_dovish_features)
        tone_s_ls.append(tone_s)
        
        no_hawkish_tone_s += 1 if tone_s > 0 else 0
        no_dovish_tone_s += 1 if tone_s < 0 else 0
    
    if no_hawkish_tone_s+no_dovish_tone_s > 0 :
        tone = (no_hawkish_tone_s-no_dovish_tone_s)/(no_hawkish_tone_s+no_dovish_tone_s)
    
    #print(no_hawkish_features_ls)
    #print(no_dovish_features_ls)
    #print(tone_s_ls)
    
    df.loc[len(df)] = [minutes['date']
                       ,tone,no_hawkish_tone_s,no_dovish_tone_s
                       ,','.join(str(x) for x in tone_s_ls)
                       ,','.join(str(x) for x in no_hawkish_features_ls)
                       ,','.join(str(x) for x in no_dovish_features_ls)
                       ,'@@@'.join(hawkish_features_ls)
                       ,'@@@'.join(dovish_features_ls)]

df.to_csv('./results/market_approach.csv')

In [4]:
df[['date','tone']].to_csv('./results/market_approach_index.csv', index=False)

In [5]:
for x, y in zip(df['hawkish_features'], df['dovish_features']) :
    print(x.split('@@@')[1])
    print(y.split('@@@')[1])
    break

일시적/VAX;외부충격/NNG,하락/NNG;일시적/VAX
원엔/NNG;환율/NNG;하락/NNG


In [6]:
dic['완만/NNG;속도/NNG;확장/NNG']

{'0': 29,
 '1': 37,
 'freq': 66,
 'w|pos': 2.324159946332357e-06,
 'w|neg': 1.6978248216578899e-06,
 'log(w|pos)': -9.570954515957652,
 'log(w|neg)': -9.884965261192164,
 'polarity score': 1.3689044456676416,
 'intensity': 1.3689044456676416,
 'lable': 1}

In [7]:
df

,date,tone,no_hawkish_tone_s,no_dovish_tone_s,tones,no_hawkish_features,no_dovish_features,hawkish_features,dovish_features
0,20050609,0.188679,63,43,"0.6,0.3333333333333333,-0.6,0,-1.0,0.0,-1.0,1....","4,2,1,0,0,1,0,4,0,0,3,1,0,2,0,0,1,1,1,3,1,1,2,...","1,1,4,0,3,1,1,0,2,0,1,0,4,1,1,0,0,0,0,1,1,1,3,...","위안/NNG,절상/NNG,절상/NNG,엔달러/NNG;환율/NNG;상승/NNG@@@일...",기대/NNG;원화/NNG;절상/NNG@@@원엔/NNG;환율/NNG;하락/NNG@@@...
1,20050707,0.207207,67,44,"-0.5,0.2,-1.0,1.0,0.0,-0.7142857142857143,-1.0...","1,3,0,1,1,1,0,0,0,2,3,7,2,1,3,3,0,1,1,3,1,1,2,...","3,2,1,0,1,6,2,0,2,1,2,2,1,2,4,0,0,2,2,1,0,0,1,...","절하/NNG@@@역외/NNG,결제자금/NNG,매입초과포지션/NNG@@@@@@재점검/...","화간/NNG,지남/VV,원화/NNG;절하/NNG;압력/NNG@@@은행/NNG;환율/..."
2,20050811,0.263158,72,42,"0.8333333333333334,1.0,1.0,-0.3333333333333333...","11,8,2,1,0,1,4,4,10,1,1,12,2,3,1,5,1,0,0,0,1,2...","1,0,0,2,0,4,2,2,3,1,0,0,4,4,2,1,1,1,0,0,3,1,4,...","위안/NNG,절상/NNG,절상/NNG,위안/NNG,절상/NNG,절상/NNG,위안/N...","이루어졌으므로/VV@@@@@@@@@유형/NNG,자본/NNG;유출/NNG;가능성/NN..."
3,20050908,0.153153,64,47,"0.3333333333333333,0.0,0.16666666666666666,0.0...","2,1,7,2,4,0,0,2,2,3,1,1,4,2,1,0,8,2,1,0,7,5,1,...","1,1,5,2,1,3,3,2,2,0,2,4,5,1,4,0,2,2,0,4,0,4,4,...","역외/NNG,환시장/NNG@@@外資/NNG@@@인상/NNG,제자리/NNG,인상/NN...","불안사태/NNG@@@투자/NNG;매우/MAG;부진/NNG@@@카트리나/NNG,카트리..."
4,20051011,0.610169,95,23,"1.0,-0.3333333333333333,-1.0,-0.71428571428571...","3,1,0,1,2,0,0,2,1,0,5,4,2,1,1,2,3,3,0,1,2,1,2,...","0,2,3,6,1,0,1,1,1,0,2,0,0,4,0,0,2,3,2,1,0,1,1,...","인상/NNG,인상/NNG,국채/NNG;발행/NNG;줄/VV@@@심각/NNG;확대/N...","@@@회복/NNG;장기/NNG;금리/NNG;상승/NNG,자본/NNG;유출/NNG;가..."
5,20051110,0.495495,83,28,"-1.0,1.0,0.0,1.0,1.0,-0.3333333333333333,0.0,1...","0,2,1,1,1,2,1,1,3,2,10,0,9,2,3,0,2,1,1,0,0,2,1...","1,0,1,0,0,4,1,0,0,1,0,0,3,3,1,0,1,0,0,1,2,1,0,...","@@@경상/NNG;수지/NNG;적자/NNG;재정/NNG;적자/NNG,적자/NNG;문...",재정/NNG;적자/NNG;경상/NNG;수지/NNG;적자/NNG@@@@@@질/XSV@...
6,20051208,0.596491,91,23,"0.0,-0.7142857142857143,0.7142857142857143,1.0...","1,1,6,4,3,4,0,1,4,0,3,4,0,0,0,1,0,1,2,1,3,1,2,...","1,6,1,0,0,0,0,1,0,0,0,2,0,0,0,0,0,0,0,2,3,0,0,...","위안/NNG@@@세계/NNG;경제/NNG;호조/NNG@@@자본거래/NNG,절상/NN...","줄어듦/VV@@@언/VV,격한/VV,불황/NNG,불황/NNG,공조/NNG,세계/NN..."
7,20060112,0.376000,86,39,"-1.0,0,0.3333333333333333,0.7777777777777778,0...","0,0,2,8,3,4,2,4,2,0,2,0,5,0,8,0,0,0,1,3,11,1,0...","1,0,1,1,1,4,1,1,0,1,2,2,1,1,2,3,0,0,0,1,2,5,1,...","@@@@@@투기거래/NNG,역외/NNG@@@위안/NNG,위안/NNG,절상/NNG,시...",연말연시/NNG@@@@@@다/VA@@@절상/NNG;확대/NNG@@@수출/NNG;위축...
8,20060209,0.775862,103,13,"0,1.0,-1.0,0.6923076923076923,1.0,0.7777777777...","0,1,0,11,2,8,0,2,12,3,0,2,4,1,1,5,1,2,1,3,2,1,...","0,0,2,2,0,1,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,...","@@@역외/NNG@@@@@@발표문/NNG,인상/NNG,인상/NNG,절상/NNG,인상...","@@@@@@외화자금/NNG,외환시장/NNG;안정/NNG@@@개/NNG,소규모/NNG..."
9,20060309,0.827586,106,10,"1.0,1.0,0.3333333333333333,0.3333333333333333,...","5,7,2,2,2,8,1,9,4,3,0,0,1,1,1,5,0,1,4,1,1,1,5,...","0,0,1,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,...","위안/NNG,위안/NNG,절상/NNG,위안/NNG,절상/NNG@@@인상/NNG,위안...",@@@@@@외화/NNG@@@좋/VA;않/VX;영향/NNG;줄/VV@@@@@@@@@@...


In [10]:
import pandas as pd
from os import listdir
from os.path import isfile, join

#news
source_folder = "./data/news/"

news = pd.read_csv(source_folder + "news_2005_preprocessed_n.csv")


In [11]:
news.head()

,media,date,title,url,news,tokens,ngram
0,연합인포맥스,2005.01.01,내년 美국채수익률 5% 넘어서기 어려울 듯,https://news.naver.com/main/read.nhn?mode=LSD&...,2005년 10년만기 미국 국채수익률이 연방준비제도이사회(FRB)의 금리인상 지속...,"만기/NNG,국채/NNG,수익률/NNG,fed/NNG,fed/NNG,금리/NNG,인...","단기/NNG;국채/NNG;수익률/NNG;상승/NNG,국채/NNG;수익률/NNG;상승..."
1,연합인포맥스,2005.01.01,[뉴욕채권-마감] 10년만기 국채수익률 작년보다 낮은 수준서 마쳐,https://news.naver.com/main/read.nhn?mode=LSD&...,2년만기 국채가격 4년래 최악의 한해 보내 10년만기 미국 국채수익률이 ...,"만기/NNG,국채/NNG,가격/NNG,최악/NNG,보내/VV,만기/NNG,국채/NN...",국채/NNG;가격/NNG;fed/NNG;금리/NNG;인상/NNG
2,연합뉴스,2005.01.01,강진군 이차보전 사업 대폭 확대,https://news.naver.com/main/read.nhn?mode=LSD&...,전남 강진군이 농가 소득보전 차원의 이차(利差)보전 사업을 대폭 확대한다. ...,"강진군/NNG,농가/NNG,소득/NNG,보전/NNG,차원/NNG,이차/NNG,利/N...",NaN
3,연합뉴스,2005.01.01,<표> 은행 금전신탁배당률.정기예금 금리(12.31일),https://news.naver.com/main/read.nhn?mode=LSD&...,)),NaN,NaN
4,연합인포맥스,2005.01.01,[뉴욕환시] `내년초 달러-엔에 주력'..달러 對엔 하락,https://news.naver.com/main/read.nhn?mode=LSD&...,2004년 마지막 거래일인 31일 뉴욕환시에서 미국 달러화는 개장초 102엔 근처...,"마지막/NNG,거래일/NNG,뉴욕/NNG,환시/NNG,달러/NNG,개장/NNG,엔/...","경상/NNG;수지/NNG;적자/NNG,유로달러/NNG;약세/NNG,유로엔/NNG;강..."
